In [1]:
import pandas as pd 
import numpy as np
import scipy.stats as sts

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

C:\anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Что есть:

train.csv +

test.csv + 
 
oil.csv + 

stores.csv ??? 

transactions.csv + 

holidays_events.csv + -> holidays

In [2]:
df = pd.read_csv('train.csv')

In [3]:
oil = pd.read_csv('oil.csv')

In [4]:
test = pd.read_csv('test.csv') 

stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')


In [191]:
holidays_events = pd.read_csv('holidays_events.csv')

### 1. Training data

In [5]:
df = df.drop('id', axis=1)
test = test.drop('id', axis=1)

In [217]:
df

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,2013-01-01,1,BABY CARE,0.000,0
2,2013-01-01,1,BEAUTY,0.000,0
3,2013-01-01,1,BEVERAGES,0.000,0
4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0
3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [6]:
display(df.head(), test.head())

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


,date,store_nbr,family,onpromotion
0,2017-08-16,1,AUTOMOTIVE,0
1,2017-08-16,1,BABY CARE,0
2,2017-08-16,1,BEAUTY,2
3,2017-08-16,1,BEVERAGES,20
4,2017-08-16,1,BOOKS,0


In [7]:
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
test['date'] = pd.to_datetime(test['date'], format = "%Y-%m-%d")
transactions['date'] = pd.to_datetime(transactions['date'], format = "%Y-%m-%d")

In [192]:
holidays_events['date'] = pd.to_datetime(holidays_events['date'], format = "%Y-%m-%d")

In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    int64         
 2   family       object        
 3   sales        float64       
 4   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 114.5+ MB


In [212]:
df.to_csv('train_fixed.csv')

In [214]:
test.to_csv('test_fixed.csv')

In [215]:
transactions.to_csv('transactions_fixed.csv')

In [213]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         28512 non-null  datetime64[ns]
 1   store_nbr    28512 non-null  int64         
 2   family       28512 non-null  object        
 3   onpromotion  28512 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 891.1+ KB


In [10]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [145]:
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB


В 1 день продукты продавались только в магазине 25. 

In [11]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


Можно заметить, что в данных по котировкам нефти есть пропуски -- рассматриваю вариант заполнить их средним от двух соседних значений. Почему? Мне кажется это разумным, потому что, если не было внезапных выбросов на сутки (что звучит нереалистично, обычно шоки все равно имеют влияние больше, чем всего на день), то нефть будет приблизительно придерживаться тех же значений, что вчера и сегодня. 

In [12]:
oil.isna().sum()

date           0
dcoilwtico    43
dtype: int64

In [13]:
oil_null = oil[oil.isna().any(axis=1)]
oil_null.head()

,date,dcoilwtico
0,2013-01-01,NaN
14,2013-01-21,NaN
34,2013-02-18,NaN
63,2013-03-29,NaN
104,2013-05-27,NaN


In [14]:
list_of_dataframes = [oil]

In [15]:
for value in oil_null['date']:
    ind = oil.index[oil.date == value]
    if ind-1 > 0:
        previous_price = oil.iloc[ind-1]
        next_price = oil.iloc[ind+1]
        if (previous_price['dcoilwtico'].isnull().any() == False) and (next_price['dcoilwtico'].isnull().any() == False):
            pp = previous_price.iloc[0]['dcoilwtico']
            np = next_price.iloc[0]['dcoilwtico']
            res = (pp + np) / 2
            bb = oil.loc[oil['date'] == value, ['date', 'dcoilwtico']]
            bb['dcoilwtico'] = res
            list_of_dataframes.append(bb)
        elif previous_price['dcoilwtico'].isnull().any() == True: 
            np = next_price.iloc[0]['dcoilwtico']
            res = np 
            bb = oil.loc[oil['date'] == value, ['date', 'dcoilwtico']]
            bb['dcoilwtico'] = res
            list_of_dataframes.append(bb)
        elif next_price['dcoilwtico'].isnull().any() == True:
            pp = previous_price.iloc[0]['dcoilwtico']
            res = pp
            bb = oil.loc[oil['date'] == value, ['date', 'dcoilwtico']]
            bb['dcoilwtico'] = res
            list_of_dataframes.append(bb)
    else:
        next_price = oil.iloc[ind+1]
        np = next_price.iloc[0]['dcoilwtico']
        res = np 
        bb = oil.loc[oil['date'] == value, ['date', 'dcoilwtico']]
        bb['dcoilwtico'] = res
        list_of_dataframes.append(bb)

In [16]:
oil_new = pd.concat(list_of_dataframes, axis=0)
oil_new 

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.140
2,2013-01-03,92.970
3,2013-01-04,93.120
4,2013-01-07,93.200
...,...,...
1079,2017-02-20,53.715
1118,2017-04-14,52.905
1149,2017-05-29,49.605
1174,2017-07-03,46.020


In [17]:
oil_n = oil_new.dropna()
oil_n

,date,dcoilwtico
1,2013-01-02,93.140
2,2013-01-03,92.970
3,2013-01-04,93.120
4,2013-01-07,93.200
5,2013-01-08,93.210
...,...,...
1079,2017-02-20,53.715
1118,2017-04-14,52.905
1149,2017-05-29,49.605
1174,2017-07-03,46.020


In [18]:
oil_n['date'] = pd.to_datetime(oil_n['date'], format = "%Y-%m-%d")

In [19]:
oil = oil_n.sort_values(by='date')
oil

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


#### УРА, красивая нефть без пропусков **получена**!!!!

In [210]:
oil.to_csv('oil_fixed.csv')

### Holidays

**Внезапно!!** Информация по holidays_events содержит праздничные даты вне нужного нам периода. Удалим! 

In [34]:
from datetime import datetime, timedelta

In [193]:
first_holiday = datetime.strptime('2013-01-01', '%Y-%m-%d')
last_holiday = datetime.strptime('2017-08-31', '%Y-%m-%d')

In [194]:
holidays = holidays_events[(holidays_events['date'] >= first_holiday) & (holidays_events['date'] <= last_holiday)]
holidays = holidays.reset_index()
holidays.drop('index', axis=1 , inplace=True)

Изучим holidays types подробнее: всего есть 
*'Holiday'* – праздничный день, скорее всего, ***магазины работать не будут или не все*; 

'Additional' – дополнительный праздничный день, приуроченный к какому-то празднику; 

'Work Day' – праздник есть, но галочкой – продажи обычные, скорее всего 

'Event' – предполагаю, что проводится какое-то мероприятие? НЕ выходной день, пример из таблицы Black Friday. во время event магазины вряд ли закрываются, так что **продажи могут подскочить**

'Bridge – если б мы знали что это такое но мы не знаем что это такое. 

'Transfer' – тут вообще надо придумать, как перезаписать. важное свойство – transfer усть только у Holiday  

по факту, если какое-то значение transfer == true, тогда надо перебрать ближайшие последующие дни, найти строку
holidays[‘type’] == ‘transfer’ и надо просто перезаписать дату из одной ячейки в другую 

--

есть гипотеза, что на резкое падение нефти в цене – люди реагируют повышенными продажами; 

перед праздниками – люди закупаются (?) 



In [204]:
holidays

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
3,2013-02-11,Holiday,National,Ecuador,Carnaval,False
4,2013-02-12,Holiday,National,Ecuador,Carnaval,False
...,...,...,...,...,...,...
282,2017-08-05,Holiday,Local,Esmeraldas,Fundacion de Esmeraldas,False
283,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
284,2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
285,2017-08-15,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [199]:
transfer_of_hds = holidays[holidays['type'] == 'Transfer']
transfer_of_hds.shape

(9, 6)

In [198]:
tr_holidays = holidays[holidays['transferred'] == True]
tr_holidays.shape

(9, 6)

In [200]:
lst_of_hds = [transfer_of_hds, tr_holidays]

hds = pd.concat(lst_of_hds, axis=0)
hds = hds.sort_values(by='date')
hds #shape = (18, 6)

,date,type,locale,locale_name,description,transferred
31,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
32,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
94,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
95,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
214,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
215,2016-05-27,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
224,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False
225,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
227,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
228,2016-08-12,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False


In [152]:
hds.iloc[0]['transferred']

True

In [108]:
hds.iloc[i+1]['date']

Timestamp('2013-10-11 00:00:00')

In [184]:
i=0
value = 'Independencia de Guayaquil'
vv = hds.loc[(hds['transferred'] == True)& (hds['description'] == value), ['date', 'type', 'locale', 'locale_name', 'description', 'transferred']]

v = vv.iloc[[0]]
v['date'] = hds.iloc[i+1]['date']
v['transferred'] = False
v

#vv['date'] = hds.iloc[i+1]['date']
#vv['date'] = hds.iloc[i+1]['date']
#vv['transferred'] = False
#vv
#v = vv.iloc[[0]]
#v
#v['date'] = hds.iloc[i+1]['date']
#v['transferred'] = False
#lst.append(v)

,date,type,locale,locale_name,description,transferred
31,2013-10-11,Holiday,National,Ecuador,Independencia de Guayaquil,False


In [168]:
vv.shape 
    

(2, 6)

In [201]:
lst = []

In [202]:
for i in range(hds.shape[0]):
    value = hds.iloc[i]['description']
    if hds.iloc[i]['transferred'] == True:
        if value in hds.iloc[i+1]['description']:
            vv = hds.loc[(hds['transferred'] == True)& (hds['description'] == value), ['date', 'type', 'locale', 'locale_name', 'description', 'transferred']]
            v = vv.iloc[[0]]
            v['date'] = hds.iloc[i+1]['date']
            v['transferred'] = False
            lst.append(v)
        elif value in hds.iloc[i-1]['description'] and i-1 > 0:
            vv = hds.loc[(hds['transferred'] == True)& (hds['description'] == value), ['date', 'type', 'locale', 'locale_name', 'description', 'transferred']]
            v = vv.iloc[[0]]
            v['date'] = hds.iloc[i+1]['date']
            v['transferred'] = False
            lst.append(v)
w = pd.concat(lst, axis=0)
w

,date,type,locale,locale_name,description,transferred
31,2013-10-11,Holiday,National,Ecuador,Independencia de Guayaquil,False
31,2014-10-10,Holiday,National,Ecuador,Independencia de Guayaquil,False
214,2016-05-27,Holiday,National,Ecuador,Batalla de Pichincha,False
225,2016-08-10,Holiday,Local,Guayaquil,Fundacion de Guayaquil,False
227,2016-08-12,Holiday,National,Ecuador,Primer Grito de Independencia,False
256,2017-01-02,Holiday,National,Ecuador,Primer dia del ano,False
262,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca,False
214,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha,False
227,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia,False


In [203]:
w.shape ## ровно столько же, сколько было transferred праздников! мы удачно всё перенесли!!! 

(9, 6)

In [205]:
lstt = [holidays, w]

In [206]:
united_holidays = pd.concat(lstt, axis=0)

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
3,2013-02-11,Holiday,National,Ecuador,Carnaval,False
4,2013-02-12,Holiday,National,Ecuador,Carnaval,False
...,...,...,...,...,...,...
227,2016-08-12,Holiday,National,Ecuador,Primer Grito de Independencia,False
256,2017-01-02,Holiday,National,Ecuador,Primer dia del ano,False
262,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca,False
214,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha,False


In [208]:
holidays = united_holidays[(united_holidays['type'] != 'Transfer') & (united_holidays['transferred'] != True)]
holidays = holidays.sort_values(by='date')
holidays

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
3,2013-02-11,Holiday,National,Ecuador,Carnaval,False
4,2013-02-12,Holiday,National,Ecuador,Carnaval,False
...,...,...,...,...,...,...
281,2017-07-25,Additional,Local,Guayaquil,Fundacion de Guayaquil,False
282,2017-08-05,Holiday,Local,Esmeraldas,Fundacion de Esmeraldas,False
227,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia,False
285,2017-08-15,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [209]:
holidays.to_csv('holidays.csv')